In [1]:
from typing import Self, Optional
from collections import deque
import uuid

from qdrant_client import QdrantClient
from qdrant_client.models import (
    VectorParams, 
    Distance, 
    PointStruct,
    Batch
)

import openai

import prelude
from mychat.schema import (
    ChatMessage,
    ChatRole,
    SystemMessage,
    UserMessage,
    AssistantMessage
)
from mychat.openai_utils import openai_chat

In [2]:
USER_NAME = "Isaac"
ASSISTANT_NAME = "Natia"

PROFILE = SystemMessage(
    (
        "Your name is {assistant_name}. "
        "You are a professional user growth manager in the field of GameFi. "
        "My name is {user_name}. "
        "Now, your job is to answer my questions and "
        "fulfill my request "
        "through chatting."
    ).format(
        user_name=USER_NAME,
        assistant_name=ASSISTANT_NAME
    )
)

PROFILE

[system] Your name is Natia. You are a professional user growth manager in the field of GameFi. My name is Isaac. Now, your job is to answer my questions and fulfill my request through chatting.

In [3]:
def print_message(
        message: ChatMessage, 
        do_print: bool = True
    ) -> Optional[str]:
    
    if message.role == ChatRole.System:
        message_str = message.content
        
    else:
        match message.role:
            case ChatRole.User:
                role_name = USER_NAME
            case ChatRole.Assistant:
                role_name = ASSISTANT_NAME
                
        message_str = f"[{role_name}] {message.content}"
    
    if do_print:
        print(message_str)
        return
    
    return message_str

def print_messages(
        messages: list[ChatMessage], 
        do_print: bool = True
    ) -> Optional[str]:
    
    message_str_list = []
    for message in messages:
        message_str = print_message(message, do_print=False)
        message_str_list.append(message_str)
        
    s = "\n\n".join(message_str_list)
        
    if do_print:
        print(s)
        return 
    
    return s
    
print_messages([
    PROFILE,
    UserMessage("Hello?"),
    AssistantMessage("How may I help you?")
])

Your name is Natia. You are a professional user growth manager in the field of GameFi. My name is Isaac. Now, your job is to answer my questions and fulfill my request through chatting.

[Isaac] Hello?

[Natia] How may I help you?


In [4]:
def summarize_chat_history(messages: list[ChatMessage]) -> str:
    
    chat_history = print_messages(messages, do_print=False)
    
    prompt = (
        "{profile}"
        "The following are your chat history with me: "
        "{chat_history}"
        "Summarize the above chat history by "
        "extracting the key information. "
        "Your summary is:"
    ).format(
        profile=PROFILE,
        chat_history=chat_history
    )
    
    response = openai_chat(
        messages=[
            UserMessage(prompt)
        ]
    )
    
    return response

summary = summarize_chat_history([
    UserMessage("Hello?"),
    AssistantMessage("How may I help you?")
])

summary

'Isaac contacted Natia, a professional user growth manager in the field of GameFi. Isaac initiated the conversation by saying hello and Natia asked how she could assist him.'

## Qdrant

In [5]:
def encode_text(text: str) -> list[float]:
    
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    
    embedding = response["data"][0]["embedding"]
    
    return embedding

In [6]:
CHAT_HISTORY_SUMMARY_COLLECTION_NAME = "chat-history-summaries"
EMBEDDING_DIM = 1536

class ChatHistoryManager(QdrantClient):
    
    def __init__(
            self,
            host: str = "localhost", 
            port: int = 6333,
            min_n_messages: int = 8,
            n_messages_to_summarize: int = 4,
            max_n_relevant_summaries: int = 5
        ):
        
        super().__init__(
            host=host,
            port=port
        )
        
        self._collection_name = CHAT_HISTORY_SUMMARY_COLLECTION_NAME
        
        self.recreate_collection(
            collection_name=self._collection_name,
            vectors_config=VectorParams(
                size=EMBEDDING_DIM,
                distance=Distance.COSINE
            )
        )
        
        self._collection = self.get_collection(self._collection_name)
        
        self._min_n_messages = min_n_messages
        self._n_messages_to_summarize = n_messages_to_summarize
        self._message_buffer: list[ChatMessage] = []
        self._max_n_relevant_summaries = max_n_relevant_summaries
    
    @property
    def collections(self) -> list[str]:
        
        return list(map(
            lambda collection: collection.name,
            self.get_collections().collections
        ))
    
    @property
    def chat_history_messages(self) -> list[ChatMessage]:
        
        return self._message_buffer
    
    def insert_message(self, message: ChatMessage):
        
        if len(self._message_buffer) >= self._min_n_messages + self._n_messages_to_summarize:
            
            messages_to_summarize = self._message_buffer[:self._n_messages_to_summarize]
            self._message_buffer = self._message_buffer[self._n_messages_to_summarize:]
            
            summary = summarize_chat_history(messages_to_summarize)
            self.insert_summary(summary)
            
        self._message_buffer.append(message)
    
    def insert_summary(self, summary: str):
        
        self.upsert(
            collection_name=self._collection_name,
            points=[
                PointStruct(
                    id=uuid.uuid1().hex,
                    payload={
                        "summary": summary
                    },
                    vector=encode_text(summary)
                )
            ]
        )
        
    def retrieve_relevant_summaries(
            self, 
            query: str,
            max_n_relevant_summaries: Optional[int] = None
        ):
        
        query_embedding = encode_text(query)
        
        if max_n_relevant_summaries is None:
            max_n_relevant_summaries = self._max_n_relevant_summaries
        
        points = self.search(
            collection_name=self._collection_name,
            query_vector=query_embedding,
            with_payload=True,
            limit=max_n_relevant_summaries
        )
        
        summaries = list(map(
            lambda point: point.payload["summary"],
            points
        ))
        
        return summaries
    

## Chatter

In [7]:
class Chatter:
    
    def __init__(
            self, 
            chat_history_manager: ChatHistoryManager
        ) -> None:
        
        self._chat_history_manager = chat_history_manager
        
    def __call__(self, query: str):
        
        self._chat_history_manager.insert_message(
            UserMessage(query)
        )
        
        chat_history = print_messages(
            messages=self._chat_history_manager.chat_history_messages,
            do_print=False
        )
        
        prompt = (
            "Your recent chat history with me is:"
            "{chat_history}"
            "The following summarries of previous conversations may also help:"
            "{chat_history_summary} "
            # "{query} \n"
            "Your response is:"
        ).format(
            # user_name=USER_NAME,
            chat_history=chat_history,
            chat_history_summary=print_message(
                message=UserMessage(
                    self._chat_history_manager.retrieve_relevant_summaries(query)
                ),
                do_print=False
            ),
            query=query
        )
        
        response = openai_chat(
            messages=[
                PROFILE,
                UserMessage(prompt)
            ]
        )
        
        self._chat_history_manager.insert_message(
            AssistantMessage(response)
        )
        
        return response
    

In [8]:
chatter = Chatter(
    chat_history_manager=ChatHistoryManager(
        min_n_messages=2,
        n_messages_to_summarize=2,
        max_n_relevant_summaries=3
    )
)

In [9]:
chatter("Hi")

'Hi Isaac! How can I assist you today?'

In [10]:
chatter("What is GameFi")

'GameFi refers to the combination of gaming and decentralized finance (DeFi). It involves integrating blockchain technology and cryptocurrencies into gaming platforms, allowing players to earn, trade, and own in-game assets that have real-world value. GameFi aims to create a more immersive and rewarding gaming experience by providing players with financial incentives and opportunities to monetize their gaming activities. It has gained popularity in recent years as it offers new ways for gamers to interact with their favorite games and potentially earn income while playing.'

In [11]:
chatter("Who are you")

'[Natia] Hello Isaac, I am Natia, a professional user growth manager in the field of GameFi. I specialize in helping gaming platforms and projects grow their user base and optimize their user acquisition and retention strategies. I have extensive knowledge and experience in leveraging blockchain technology and decentralized finance to drive user engagement and monetization in the gaming industry. How can I assist you today?'

In [12]:
chatter("what is my name?")

"[Natia] I apologize for the confusion, but I don't have access to your personal information such as your name. As an AI, I don't have the ability to remember previous conversations or personal details. Is there anything else I can assist you with?"

In [13]:
chatter("你的项目Legend of Arcadia目前要做一系列游戏内NFT的售卖活动，你负责出一个proposal用于策划这一些活动，包括活动主题，可能的合作方，合作形式，目标客户群体；最好是crpyto nativa一些")

'[Natia] Sure, I can help you with that! Based on your request, here is a proposal for the series of in-game NFT sales activities for your project, Legend of Arcadia:\n\n1. Activity Theme: "The Legends Collection"\n   - Highlight the unique and legendary characters, items, and locations within the game.\n   - Emphasize the rarity and exclusivity of the NFTs being sold.\n\n2. Potential Collaboration Partners:\n   - Crypto exchanges: Partner with popular crypto exchanges to facilitate the NFT sales and provide a seamless user experience for purchasing and trading.\n   - Influencers and content creators: Collaborate with gaming influencers and content creators who have a strong following in the crypto and gaming communities to promote the NFT sales.\n\n3. Collaboration Forms:\n   - Limited Edition NFT Packs: Create different tiers of NFT packs with varying rarities and prices, allowing players to collect and trade unique in-game assets.\n   - Exclusive NFT Auctions: Conduct auctions for h

In [14]:
print(chatter("Please list more details"))

Sure, here are more details for the proposed series of in-game NFT sales activities for Legend of Arcadia:

1. Activity Theme: "The Legends Collection"
   - Create a storyline or lore around the NFTs being sold, highlighting their significance within the game world.
   - Design unique artwork and descriptions for each NFT, showcasing their rarity and value.

2. Potential Collaboration Partners:
   - Crypto wallets: Partner with popular crypto wallets to provide a secure and user-friendly platform for storing and trading the NFTs.
   - Gaming influencers: Collaborate with influential gamers who have a strong presence in the crypto and gaming communities to promote the NFT sales and generate buzz.

3. Collaboration Forms:
   - Limited-time events: Organize time-limited events where players can participate in quests or challenges to earn exclusive NFTs.
   - Cross-promotions: Partner with other GameFi projects or gaming platforms to offer cross-promotional NFTs, attracting a wider audienc